In [10]:
#import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time
from sklearn.preprocessing import StandardScaler
import numpy as np # for math and arrays
import pandas as pd


titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')
inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = tf.keras.layers.Concatenate()(list(numeric_inputs.values()))
norm = tf.keras.layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = tf.keras.layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
  one_hot = tf.keras.layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

preprocessed_inputs_cat = tf.keras.layers.Concatenate()(preprocessed_inputs)

titanic_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

titanic_features_dict = {name: np.array(value) 
                         for name, value in titanic_features.items()}

features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}
titanic_preprocessing(features_dict)

def titanic_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
  ])

  preprocessed_inputs = titanic_preprocessing(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.optimizers.Adam(),metrics=['accuracy'])
  return model

titanic_model = titanic_model(titanic_preprocessing, inputs)

titanic_model.fit(x=titanic_features_dict, y=titanic_labels, epochs=1)



20/20 [==============================] - 1s 2ms/step - loss: 0.6738 - accuracy: 0.6124


In [11]:
1+1

2

In [12]:
#import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time
from sklearn.preprocessing import StandardScaler
import numpy as np # for math and arrays
import pandas as pd


titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')
inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = tf.keras.layers.Concatenate()(list(numeric_inputs.values()))
norm = tf.keras.layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = tf.keras.layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
  one_hot = tf.keras.layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

preprocessed_inputs_cat = tf.keras.layers.Concatenate()(preprocessed_inputs)

titanic_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

titanic_features_dict = {name: np.array(value) 
                         for name, value in titanic_features.items()}

features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}
titanic_preprocessing(features_dict)

def return_compiled_model(how_many_dense_layers:int, layer_size:int,preprocessing_head, inputs) ->tf.keras.models.Sequential:
    body = tf.keras.models.Sequential()
    body.add(tf.keras.layers.Dense(64))
    for i in range(how_many_dense_layers):
        body.add(tf.keras.layers.Dense(layer_size))
    body.add(tf.keras.layers.Dense(1))  
    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)

    model.compile(optimizer='adam',
            loss='BinaryCrossentropy',#'categorical_crossentropy',
            metrics=['accuracy'])
    return model

def create_model_short_name(how_many_dense_layers:int, layer_size:int) ->str:
    return str(how_many_dense_layers)+'layers_'+str(layer_size)+'nodes_'

all_model_specs = [{"layers": 0 ,"nodes": 0 }]
layers = [1,2,3,4]
nodes = [16,32,64,128,256,256*2,256*4]
for layer in layers:
    for node in nodes:
        all_model_specs.append({"layers": layer ,"nodes": node })
print("model spec combinations:",len(all_model_specs))

for model_spec in tqdm(all_model_specs):
    log_name = create_model_short_name(model_spec['layers'],model_spec['nodes'])
    log_dir = "logs_titanic03_/fit/" + log_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model = return_compiled_model(model_spec['layers'],model_spec['nodes'],titanic_preprocessing, inputs)
    with tf.device('/CPU:0'): # it can be with '/CPU:0'
        history = model.fit(titanic_features_dict, titanic_labels, epochs=100, shuffle=True,  verbose=0
        
        , callbacks=[tensorboard_callback] ) #validation_data = (x_valid,y_valid),



model spec combinations: 29


100%|██████████| 29/29 [14:46<00:00, 30.58s/it]
